# Lending Tree Credit Risk

### Dependencies and data

In [16]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTEENN
from imblearn.ensemble import EasyEnsembleClassifier, BalancedRandomForestClassifier

# print(mpl.style.available)
mpl.style.use('Solarize_Light2')
%matplotlib inline

In [17]:
# Inspect top rows in data
with open('lt-credit-risk/data/loans_1q19.csv') as f:
    for i in range(5):
        print(f.readline()[:100])

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade"
"","","20000","20000","20000"," 60 months"," 17.19%","499.1","C","C5","Front desk supervisor","6 yea
"","","21225","21225","21225"," 60 months"," 14.74%","502.05","C","C2","ceo","10+ years","MORTGAGE",
"","","5000","5000","5000"," 36 months"," 17.97%","180.69","D","D1","","n/a","MORTGAGE","62000","Not


In [18]:
# Data
df = pd.read_csv('lt-credit-risk/data/loans_1q19.csv', skiprows=1, low_memory=False)
print(df.shape)
df.head(3)

(115677, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,60 months,17.19%,499.10,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,21225.0,21225.0,21225.0,60 months,14.74%,502.05,C,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000.0,5000.0,5000.0,36 months,17.97%,180.69,D,D1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


### Drop columns

In [19]:
# Drop columns with more than half its values missing
df.dropna(axis=1, thresh=df.shape[0]//2, inplace=True)
df.shape

(115677, 101)

In [20]:
# Drop constant columns
const_cols = df.nunique()[df.nunique() < 2].index # cols w/ 1 unique val
df.drop(const_cols, axis=1, inplace=True)
df.shape

(115677, 91)

### Non-numeric columns

In [49]:
# Inspect non-numeric columns
df_num = df.copy() # make a copy
obj_cols = df_num.dtypes[df_num.dtypes == object].index
df[obj_cols].head(3)

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,...,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type
0,60 months,17.19%,C,C5,Front desk supervisor,6 years,RENT,Source Verified,Mar-2019,Issued,...,Debt consolidation,958xx,CA,Sep-2006,19.7%,w,NaN,Apr-2019,Apr-2019,Individual
1,60 months,14.74%,C,C2,ceo,10+ years,MORTGAGE,Not Verified,Mar-2019,Issued,...,Credit card refinancing,956xx,CA,Sep-1994,87.7%,w,NaN,Apr-2019,Apr-2019,Individual
2,36 months,17.97%,D,D1,NaN,NaN,MORTGAGE,Not Verified,Mar-2019,Issued,...,Home improvement,320xx,FL,Nov-1987,77.5%,w,NaN,Apr-2019,Apr-2019,Individual


In [50]:
""" String manipulation """

# Convert `term` to numeric
df_num['term'] = df['term'].str.replace(' months', '').astype(float)

# Convert `int_rate` to numeric
df_num['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)

# Convert `sub_grade` to numeric
df_num['sub_grade'] = df['sub_grade'].str.slice(1).astype(float)

# Convert `emp_length` to numeric
df_num['emp_length'] = df['emp_length'].str.extract('(\d+)').astype(float)

# Convert `revol_util` to numeric
df_num['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)

df_num.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,20000.0,20000.0,20000.0,60.0,17.19,499.10,C,5.0,Front desk supervisor,6.0,...,15.0,0.0,5.0,98.0,12.5,0.0,75824.0,31546.0,33800.0,21524.0
1,21225.0,21225.0,21225.0,60.0,14.74,502.05,C,2.0,ceo,10.0,...,14.0,0.0,2.0,100.0,50.0,1.0,747075.0,209426.0,53500.0,128175.0
2,5000.0,5000.0,5000.0,36.0,17.97,180.69,D,1.0,NaN,NaN,...,8.0,1.0,1.0,66.7,50.0,0.0,255738.0,31615.0,9400.0,39938.0


In [51]:
""" Datetime manipulation """

# Create a new column for `issue_d` as numeric type
df_num['issue_month'] = pd.to_datetime(df['issue_d']).dt.month

# Create a new column for `earliest_cr_line` as numeric type
df_num['earliest_cr_line'] = pd.to_datetime(df_num['earliest_cr_line']) # convert to dt
youngest_cr = df_num['earliest_cr_line'].max() # latest date in data
df_num['oldest_cr_age'] = (youngest_cr - df_num['earliest_cr_line']).dt.days # oldest credit age

# Create a new column for `last_pymnt_d` as numeric type
df_num['last_payment_month'] = pd.to_datetime(df['last_pymnt_d']).dt.month

# Create a new column for 'last_credit_pull_d' as numeric type
df_num['last_credit_pull_month'] = pd.to_datetime(df['last_credit_pull_d']).dt.month
df_num['last_credit_pull_month'] = df_num['last_credit_pull_month'].replace(12, 0) # set Dec 2018 as month 0

df_num.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,issue_month,oldest_cr_age,last_payment_month,last_credit_pull_month
0,20000.0,20000.0,20000.0,60.0,17.19,499.10,C,5.0,Front desk supervisor,6.0,...,12.5,0.0,75824.0,31546.0,33800.0,21524.0,3.0,3440.0,NaN,4.0
1,21225.0,21225.0,21225.0,60.0,14.74,502.05,C,2.0,ceo,10.0,...,50.0,1.0,747075.0,209426.0,53500.0,128175.0,3.0,7823.0,NaN,4.0
2,5000.0,5000.0,5000.0,36.0,17.97,180.69,D,1.0,NaN,NaN,...,50.0,0.0,255738.0,31615.0,9400.0,39938.0,3.0,10319.0,NaN,4.0


In [52]:
""" Numeric mapping """

# Convert `grade` to numeric
grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7} # str to num mapping
df_num['grade'] = df['grade'].map(grade_mapping).astype(float)

# Convert `verification_state` to numeric
df_num['verification_status'] = df['verification_status'].str.replace('Source ', '') # combine verified classes
veri_mapping = {'Not Verified': 0, 'Verified': 1} # str to num mapping
df_num['verification_status'] = df['verification_status'].map(veri_mapping).astype(float)

# Create a new column for `initial_list_status` as numeric type
init_mapping = {'f': 0, 'w': 1} # str to num mapping
df_num['whole_loan'] = df['initial_list_status'].map(init_mapping).astype(float)

# Create a new column for `application_type` as numeric type
app_mapping = {'Individual': 0, 'Joint App': 1} # str to num mapping
df_num['joint_app'] = df['application_type'].map(app_mapping).astype(float)


df_num.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,issue_month,oldest_cr_age,last_payment_month,last_credit_pull_month,whole_loan,joint_app
0,20000.0,20000.0,20000.0,60.0,17.19,499.10,3.0,5.0,Front desk supervisor,6.0,...,75824.0,31546.0,33800.0,21524.0,3.0,3440.0,NaN,4.0,1.0,0.0
1,21225.0,21225.0,21225.0,60.0,14.74,502.05,3.0,2.0,ceo,10.0,...,747075.0,209426.0,53500.0,128175.0,3.0,7823.0,NaN,4.0,1.0,0.0
2,5000.0,5000.0,5000.0,36.0,17.97,180.69,4.0,1.0,NaN,NaN,...,255738.0,31615.0,9400.0,39938.0,3.0,10319.0,NaN,4.0,1.0,0.0


In [53]:
# Drop redundant cols
cols_to_drop = ['title', 'zip_code', 'issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'initial_list_status', 'application_type']
df_num.drop(cols_to_drop, axis=1, inplace=True)
df_num.shape

(115677, 88)

In [58]:
# Check unique values of object columns
obj_cols2 = []
for col in obj_cols:
    if col in df_num.columns and df_num[col].dtype == object:
        obj_cols2.append(col)
        print(col)
        print(df_num[col].unique())
        print()

emp_title
['Front desk supervisor' 'ceo' nan ... 'President - North America'
 'Estimator/Supervisor' 'sr register csa']

home_ownership
['RENT' 'MORTGAGE' 'OWN' 'ANY' 'NONE' nan]

loan_status
['Issued' 'Fully Paid' 'Current' 'Charged Off' 'In Grace Period'
 'Late (16-30 days)' 'Late (31-120 days)' nan]

purpose
['debt_consolidation' 'credit_card' 'home_improvement' 'medical' 'other'
 'car' 'major_purchase' 'small_business' 'house' 'moving' 'vacation'
 'renewable_energy' nan]

addr_state
['CA' 'FL' 'PA' 'AZ' 'IL' 'GA' 'MI' 'TX' 'NY' 'OH' 'SC' 'MD' 'MO' 'LA'
 'WA' 'AR' 'WI' 'MN' 'IN' 'OK' 'VA' 'NC' 'CT' 'TN' 'WY' 'OR' 'MA' 'KY'
 'NJ' 'NM' 'ND' 'AL' 'UT' 'ID' 'KS' 'CO' 'MT' 'NH' 'NV' 'MS' 'HI' 'ME'
 'AK' 'DC' 'WV' 'DE' 'NE' 'SD' 'RI' 'VT' nan]



In [60]:
# Inspect object columns
df_num[obj_cols2].head(3)

,emp_title,home_ownership,loan_status,purpose,addr_state
0,Front desk supervisor,RENT,Issued,debt_consolidation,CA
1,ceo,MORTGAGE,Issued,credit_card,CA
2,NaN,MORTGAGE,Issued,home_improvement,FL
